In [ ]:
# ===============================
# Setup: Install Packages
# ===============================
!pip install -q \
  "transformers>=4.41,<5" \
  "datasets==2.19.1" \
  "peft==0.10.0" \
  "accelerate>=0.34.2" \
  "bitsandbytes>=0.43.3" \
  "evaluate>=0.4.2" \
  "rouge_score>=0.1.2" \
  "scikit-learn" \
  "openpyxl" \
  "pandas" \
  "xlsxwriter" \
  "bert-score>=0.3.13"


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 17.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.3.1 which is incompatible.


In [2]:
import torch, sys, subprocess
mm = ".".join(torch.__version__.split(".")[:2])
triton_by_torch = {"2.5":"3.2.0","2.4":"3.0.0","2.3":"2.3.1","2.2":"2.2.0"}
target = triton_by_torch.get(mm, "3.2.0")
print(f"Torch {torch.__version__} → Installing Triton {target}")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", f"triton=={target}"])

Torch 2.8.0+cu126 → Installing Triton 3.2.0


0

In [ ]:
# ===============================
# Import packages & login
# ===============================
from google.colab import drive
drive.mount('/content/drive')

import os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"  # or ":16:8"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


import torch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cuda.matmul.allow_tf32 = False  # avoid TF32
torch.set_float32_matmul_precision("high")     # be strict
try:
    torch.use_deterministic_algorithms(True, warn_only=True)
except Exception:
    pass



import random, numpy as np
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(SEED)

import re
import  pandas as pd
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    TrainingArguments, Trainer, set_seed
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from huggingface_hub import login

# --------------- Hugging Face token ---------------
os.environ["HF_TOKEN"] = "YOUR_TOKEN_HERE"
login(os.environ["HF_TOKEN"])

# --------------- Reproducibility ---------------
set_seed(42)

Mounted at /content/drive


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
# =========================================================
# CNN/DailyMail → ONE Excel with FT+BASE side-by-side
# - Greedy decoding
# - Keep ORIGINAL prompts; strip "Here is a summary..." AFTER loading CSVs
# - BERTScore P/R/F1 & ROUGE-1/2/Lsum P/R/F1 (per-example + aggregate)
# - Adds: per-example Reference Length + mean/median Reference Length in overview sheets
# - Sheets (order):
#   1) Results_Overview (F1-only + length stats incl. reference)
#   2) All_Metrics_PRF (full P/R/F1 + lengths incl. reference stats)
#   3) Paired_Summaries (articles + reference + FT/BASE summaries + compact F1s + lengths)
#   4) Per_Example_PRF (per-example P/R/F1 + BERTScore)
# =========================================================

import os, json, gc, re
from datetime import datetime

import torch
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.backends.cuda.matmul.allow_tf32 = False
torch.set_float32_matmul_precision("high")
try:
    torch.use_deterministic_algorithms(True, warn_only=True)
except Exception:
    pass

import tqdm
import pandas as pd
from datasets import load_dataset
import evaluate
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from rouge_score import rouge_scorer

# ── Minimal post-processing patch to strip preface ──────
_PREFACE_RE = re.compile(
    r'(?i)^\s*(?:here\s+is|here\'?s)\s+(?:a\s+)?(?:'
    r'(?:\d+\s*[-–]\s*\d+\s*sentence(?:s)?\s+summary\s+of\s+the\s+article)'  # "2-3 sentence summary of the article"
    r'|'
    r'(?:summary\s+of\s+the\s+article\s+in\s+\d+\s*[-–]\s*\d+\s*sentences?)'  # "summary of the article in 2-3 sentences"
    r')\s*:\s*\n?'
)

def strip_summary_preface(s: str) -> str:
    if not isinstance(s, str):
        return ""
    return _PREFACE_RE.sub("", s.strip(), count=1)

# ── Paths (edit if needed) ──────────────────────────────
BASE_PATH = r"/content/drive/My Drive/associations-ANLP"
RUN_DIR    = os.path.join(BASE_PATH, r"full_llama3_8b_system_prompt_lora_SFT_SWOW_tgt_qkvo_tr7194c_val899c_r16_a32_do0p1_lr0.0001_bs16_ga4")
MODEL_FT   = f"{RUN_DIR}/merged_model"
MODEL_BASE = "meta-llama/Meta-Llama-3-8B-Instruct"

SAVE_DIR = os.path.join(BASE_PATH, r"data/models_summarization")
os.makedirs(SAVE_DIR, exist_ok=True)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# ── Device & dtype ──────────────────────────────────────
if torch.cuda.is_available():
    dev_name = torch.cuda.get_device_name(0)
    cap_major, cap_minor = torch.cuda.get_device_capability(0)
    total_mem_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
else:
    dev_name, cap_major, cap_minor, total_mem_gb = "CPU", 0, 0, 0

AMPERE_PLUS = cap_major >= 8
compute_dtype = torch.bfloat16 if AMPERE_PLUS else torch.float16

MAXLEN_LADDER = (3600, 3000, 2200) if (AMPERE_PLUS or total_mem_gb >= 30) else (3000, 2200, 1600)
if total_mem_gb >= 30:
    BS_LADDER = [32, 25, 16, 12, 8, 4, 2, 1]
elif total_mem_gb >= 22:
    BS_LADDER = [8, 4, 3, 2, 1]
elif total_mem_gb >= 14:
    BS_LADDER = [8, 4, 3, 2, 1]
else:
    BS_LADDER = [2, 1] if torch.cuda.is_available() else [1]

print(f"Device: {dev_name} (cc {cap_major}.{cap_minor}) | dtype={compute_dtype} | "
      f"maxlen ladder={MAXLEN_LADDER} | batch ladder={BS_LADDER}")

# ── Data & metrics ──────────────────────────────────────
ds = load_dataset("ccdv/cnn_dailymail", "3.0.0", split="test", trust_remote_code=True)

# BERTScore config
bertscore = evaluate.load("bertscore")
bs_model = "microsoft/deberta-xlarge-mnli" if torch.cuda.is_available() else "roberta-large"
BS_BATCH_SIZE = 32  # lower if OOM/slow

# For lengths, use BASE tokenizer as a consistent yardstick
length_tokenizer = AutoTokenizer.from_pretrained(MODEL_BASE, use_fast=True)

def _load_model_and_tokenizer(model_id_or_path):
    use_bnb = torch.cuda.is_available()
    bnb_cfg = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=compute_dtype,
    ) if use_bnb else None

    tok = AutoTokenizer.from_pretrained(model_id_or_path, use_fast=True)
    if tok.pad_token_id is None and tok.eos_token_id is not None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "left"

    load_kwargs = dict(device_map="auto", low_cpu_mem_usage=True)
    if bnb_cfg is not None:
        load_kwargs["quantization_config"] = bnb_cfg
    else:
        load_kwargs["torch_dtype"] = compute_dtype

    mdl = AutoModelForCausalLM.from_pretrained(model_id_or_path, **load_kwargs)
    mdl.eval()
    return tok, mdl

def build_prompt(tok, article: str) -> str:
    # ORIGINAL prompt (unchanged)
    messages = [
        {"role": "system", "content": "You are a helpful assistant that writes concise news summaries."},
        {"role": "user", "content": f"Summarize the following article in 2-3 sentences.\n\nARTICLE:\n{article}\n\nSummary:"}
    ]
    return tok.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

@torch.inference_mode()
def _generate_batch(tok, mdl, prompts, max_len):
    inputs = tok(prompts, return_tensors="pt", padding=True, truncation=True, max_length=max_len)
    inputs = {k: v.to(mdl.device) for k, v in inputs.items()}
    out = mdl.generate(
        **inputs, max_new_tokens=500, do_sample=False,  # Greedy
        eos_token_id=tok.eos_token_id, pad_token_id=tok.pad_token_id,
    )
    attn = inputs["attention_mask"]
    L_max = inputs["input_ids"].shape[1]
    left_pad = (tok.padding_side == "left")
    gens = []
    for i in range(out.shape[0]):
        start = L_max if left_pad else int(attn[i].sum().item())
        text = "" if start >= out.shape[1] else tok.decode(out[i, start:], skip_special_tokens=True).strip()
        gens.append(text)
    del out, inputs
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    return gens

def batched_summarize_to_csv(tok, mdl, articles_iterable, batch_ladder, maxlen_ladder, tmp_csv_path, flush_every=200):
    buffer_rows, current_bs_index, current_len_index = [], 0, 0
    total = len(articles_iterable)
    pbar = tqdm.tqdm(total=total, desc="Generating")
    pd.DataFrame(columns=["id","article","reference","summary"]).to_csv(tmp_csv_path, index=False, encoding="utf-8")
    idx = 0
    while idx < total:
        bs = batch_ladder[current_bs_index]; max_len = maxlen_ladder[current_len_index]
        end = min(idx + bs, total)
        batch_ids = list(range(idx, end))
        batch_articles = [articles_iterable[i]["article"] for i in batch_ids]
        batch_refs     = [articles_iterable[i]["highlights"] for i in batch_ids]
        batch_prompts  = [build_prompt(tok, a) for a in batch_articles]
        try:
            batch_preds = _generate_batch(tok, mdl, batch_prompts, max_len)
            for i_id, a, r, g in zip(batch_ids, batch_articles, batch_refs, batch_preds):
                buffer_rows.append((i_id, a, r, g))
            idx = end; pbar.update(len(batch_preds))
            if len(buffer_rows) >= flush_every:
                pd.DataFrame(buffer_rows, columns=["id","article","reference","summary"]).to_csv(
                    tmp_csv_path, mode="a", header=False, index=False, encoding="utf-8")
                buffer_rows = []
        except torch.cuda.OutOfMemoryError:
            if torch.cuda.is_available(): torch.cuda.empty_cache()
            gc.collect()
            if current_bs_index < len(batch_ladder) - 1: current_bs_index += 1
            elif current_len_index < len(maxlen_ladder) - 1: current_len_index += 1; current_bs_index = 0
            else:
                for i_id, a, r in zip(range(idx, end), batch_articles, batch_refs):
                    buffer_rows.append((i_id, a, r, "[OOM]"))
                idx = end; pbar.update(end-idx)
    pbar.close()
    if buffer_rows:
        pd.DataFrame(buffer_rows, columns=["id","article","reference","summary"]).to_csv(
            tmp_csv_path, mode="a", header=False, index=False, encoding="utf-8")

def run_one_model_to_csv(label, model_id_or_path, tmp_csv_path):
    print(f"\n\n================= Running {label} =================")
    tok, mdl = _load_model_and_tokenizer(model_id_or_path)
    batched_summarize_to_csv(tok, mdl, ds, BS_LADDER, MAXLEN_LADDER, tmp_csv_path, flush_every=200)
    print(f"Streaming summaries → {tmp_csv_path}")
    try: del tok, mdl
    except: pass
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    gc.collect(); print(f"Freed memory after {label}")

def token_len(text: str) -> int:
    if not isinstance(text, str) or not text: return 0
    return len(length_tokenizer.encode(text, add_special_tokens=False))

def rouge_prf_lists(preds, refs, types=("rouge1","rouge2","rougeLsum")):
    """Compute per-example ROUGE P/R/F for requested types."""
    scorer = rouge_scorer.RougeScorer(list(types), use_stemmer=True)
    out = {t: {"p": [], "r": [], "f": []} for t in types}
    for p, r in tqdm.tqdm(zip(preds, refs), total=len(preds), desc="ROUGE per-example"):
        p = p if isinstance(p, str) else ""
        r = r if isinstance(r, str) else ""
        scores = scorer.score(r, p)  # target, prediction
        for t in types:
            s = scores[t]
            out[t]["p"].append(float(s.precision))
            out[t]["r"].append(float(s.recall))
            out[t]["f"].append(float(s.fmeasure))
    return out

def mean(x): return float(pd.Series(x).mean()) if len(x) else 0.0

# ── Execute both runs → combine → score ────────────────
ts = datetime.now().strftime("%Y%m%d_%H%M%S")
tmp_ft_csv   = os.path.join(SAVE_DIR, f"tmp_cnn_dm_FT_{ts}.csv")
tmp_base_csv = os.path.join(SAVE_DIR, f"tmp_cnn_dm_BASE_{ts}.csv")
combined_xlsx    = os.path.join(SAVE_DIR, f"cnn_dm_FT_BASE_combined_{ts}.xlsx")

try:
    run_one_model_to_csv("FT",   MODEL_FT,   tmp_ft_csv)
    run_one_model_to_csv("BASE", MODEL_BASE, tmp_base_csv)

    # Load temp CSVs
    ft_df   = pd.read_csv(tmp_ft_csv)
    base_df = pd.read_csv(tmp_base_csv)

    # strip boilerplate if present (safe for both; no-op unless it matches)
    ft_df["summary"]   = ft_df["summary"].apply(strip_summary_preface)
    base_df["summary"] = base_df["summary"].apply(strip_summary_preface)

    assert len(ft_df) == len(base_df), "FT and BASE CSVs have different lengths!"

    merged = ft_df.merge(base_df[["id","summary"]], on="id", suffixes=("", "_base"))
    merged.rename(columns={"summary":"ft_summary","summary_base":"base_summary"}, inplace=True)

    # Per-example lengths
    merged["ft_len"]    = [token_len(t) for t in merged["ft_summary"].tolist()]
    merged["base_len"]  = [token_len(t) for t in merged["base_summary"].tolist()]
    merged["ref_len"]   = [token_len(t) for t in merged["reference"].tolist()]

    # ---- BERTScore per-example (FT & BASE) --------------------------------
    device = "cuda" if torch.cuda.is_available() else "cpu"
    ft_bs = bertscore.compute(
        predictions=merged["ft_summary"].tolist(),
        references=merged["reference"].tolist(),
        model_type=bs_model, lang="en", rescale_with_baseline=True,
        batch_size=BS_BATCH_SIZE, device=device
    )
    base_bs = bertscore.compute(
        predictions=merged["base_summary"].tolist(),
        references=merged["reference"].tolist(),
        model_type=bs_model, lang="en", rescale_with_baseline=True,
        batch_size=BS_BATCH_SIZE, device=device
    )
    merged["ft_bs_p"]  = [float(x) for x in ft_bs["precision"]]
    merged["ft_bs_r"]  = [float(x) for x in ft_bs["recall"]]
    merged["ft_bs_f1"] = [float(x) for x in ft_bs["f1"]]
    merged["base_bs_p"]  = [float(x) for x in base_bs["precision"]]
    merged["base_bs_r"]  = [float(x) for x in base_bs["recall"]]
    merged["base_bs_f1"] = [float(x) for x in base_bs["f1"]]

    # ---- ROUGE per-example P/R/F for 1/2/Lsum (FT & BASE) -----------------
    ft_r = rouge_prf_lists(merged["ft_summary"].tolist(),   merged["reference"].tolist())
    ba_r = rouge_prf_lists(merged["base_summary"].tolist(), merged["reference"].tolist())

    # Attach compact F1s for the paired sheet
    for t, col in [("rouge1","r1"), ("rouge2","r2"), ("rougeLsum","rlsum")]:
        merged[f"ft_{col}_f1"]   = ft_r[t]["f"]
        merged[f"base_{col}_f1"] = ba_r[t]["f"]

    # Build per-example metrics sheet (all P/R/F)
    metrics_df = pd.DataFrame({
        "id": merged["id"],
        # FT ROUGE
        "ft_r1_p": ft_r["rouge1"]["p"], "ft_r1_r": ft_r["rouge1"]["r"], "ft_r1_f1": ft_r["rouge1"]["f"],
        "ft_r2_p": ft_r["rouge2"]["p"], "ft_r2_r": ft_r["rouge2"]["r"], "ft_r2_f1": ft_r["rouge2"]["f"],
        "ft_rlsum_p": ft_r["rougeLsum"]["p"], "ft_rlsum_r": ft_r["rougeLsum"]["r"], "ft_rlsum_f1": ft_r["rougeLsum"]["f"],
        # FT BERTScore
        "ft_bs_p": merged["ft_bs_p"], "ft_bs_r": merged["ft_bs_r"], "ft_bs_f1": merged["ft_bs_f1"],
        # BASE ROUGE
        "base_r1_p": ba_r["rouge1"]["p"], "base_r1_r": ba_r["rouge1"]["r"], "base_r1_f1": ba_r["rouge1"]["f"],
        "base_r2_p": ba_r["rouge2"]["p"], "base_r2_r": ba_r["rouge2"]["r"], "base_r2_f1": ba_r["rouge2"]["f"],
        "base_rlsum_p": ba_r["rougeLsum"]["p"], "base_rlsum_r": ba_r["rougeLsum"]["r"], "base_rlsum_f1": ba_r["rougeLsum"]["f"],
        # BASE BERTScore
        "base_bs_p": merged["base_bs_p"], "base_bs_r": merged["base_bs_r"], "base_bs_f1": merged["base_bs_f1"],
    })

    # Aggregate metrics (means)
    def mean_series(s): return round(float(pd.Series(s).mean()), 4)
    final_tbl = pd.DataFrame([
        {
            "model": "FT",
            "r1_p": mean_series(ft_r["rouge1"]["p"]), "r1_r": mean_series(ft_r["rouge1"]["r"]), "r1_f1": mean_series(ft_r["rouge1"]["f"]),
            "r2_p": mean_series(ft_r["rouge2"]["p"]), "r2_r": mean_series(ft_r["rouge2"]["r"]), "r2_f1": mean_series(ft_r["rouge2"]["f"]),
            "rlsum_p": mean_series(ft_r["rougeLsum"]["p"]), "rlsum_r": mean_series(ft_r["rougeLsum"]["r"]), "rlsum_f1": mean_series(ft_r["rougeLsum"]["f"]),
            "bertscore_p": mean_series(merged["ft_bs_p"]), "bertscore_r": mean_series(merged["ft_bs_r"]), "bertscore_f1": mean_series(merged["ft_bs_f1"]),
        },
        {
            "model": "BASE",
            "r1_p": mean_series(ba_r["rouge1"]["p"]), "r1_r": mean_series(ba_r["rouge1"]["r"]), "r1_f1": mean_series(ba_r["rouge1"]["f"]),
            "r2_p": mean_series(ba_r["rouge2"]["p"]), "r2_r": mean_series(ba_r["rouge2"]["r"]), "r2_f1": mean_series(ba_r["rouge2"]["f"]),
            "rlsum_p": mean_series(ba_r["rougeLsum"]["p"]), "rlsum_r": mean_series(ba_r["rougeLsum"]["r"]), "rlsum_f1": mean_series(ba_r["rougeLsum"]["f"]),
            "bertscore_p": mean_series(merged["base_bs_p"]), "bertscore_r": mean_series(merged["base_bs_r"]), "bertscore_f1": mean_series(merged["base_bs_f1"]),
        }
    ])

    # Paired sheet (compact): summaries + lengths + F1s
    paired_cols = [
        "id", "article", "reference", "ref_len",
        "ft_summary", "ft_len", "ft_r1_f1", "ft_r2_f1", "ft_rlsum_f1", "ft_bs_f1",
        "base_summary", "base_len", "base_r1_f1", "base_r2_f1", "base_rlsum_f1", "base_bs_f1",
    ]
    paired_df = merged[paired_cols]

    # --- Add mean/median summary lengths (token counts) + reference stats ---
    len_stats = pd.DataFrame({
        "model": ["FT", "BASE"],
        "len_mean":   [round(float(merged["ft_len"].mean()), 2),
                       round(float(merged["base_len"].mean()), 2)],
        "len_median": [round(float(merged["ft_len"].median()), 2),
                       round(float(merged["base_len"].median()), 2)],
    })
    final_tbl = len_stats.merge(final_tbl, on="model", how="right")

    # Reference length stats (same for both model rows)
    ref_len_mean   = round(float(merged["ref_len"].mean()), 2)
    ref_len_median = round(float(merged["ref_len"].median()), 2)
    final_tbl["ref_len_mean"]   = ref_len_mean
    final_tbl["ref_len_median"] = ref_len_median

    # ===== Friendly column names for Excel =====
    # Overview (F1-only) + reference stats
    summary_f1_tbl = final_tbl[[
        "model","len_mean","len_median","ref_len_mean","ref_len_median",
        "r1_f1","r2_f1","rlsum_f1","bertscore_f1"
    ]].rename(columns={
        "model":"Model",
        "len_mean":"Mean Summary Length (tokens)",
        "len_median":"Median Summary Length (tokens)",
        "ref_len_mean":"Reference Mean Length (tokens)",
        "ref_len_median":"Reference Median Length (tokens)",
        "r1_f1":"ROUGE-1 F1",
        "r2_f1":"ROUGE-2 F1",
        "rlsum_f1":"ROUGE-Lsum F1",
        "bertscore_f1":"BERTScore F1",
    })

    # Full metrics (P/R/F1) + reference stats
    final_tbl_named = final_tbl.rename(columns={
        "model":"Model",
        "len_mean":"Mean Summary Length (tokens)",
        "len_median":"Median Summary Length (tokens)",
        "ref_len_mean":"Reference Mean Length (tokens)",
        "ref_len_median":"Reference Median Length (tokens)",
        "r1_p":"ROUGE-1 P", "r1_r":"ROUGE-1 R", "r1_f1":"ROUGE-1 F1",
        "r2_p":"ROUGE-2 P", "r2_r":"ROUGE-2 R", "r2_f1":"ROUGE-2 F1",
        "rlsum_p":"ROUGE-Lsum P", "rlsum_r":"ROUGE-Lsum R", "rlsum_f1":"ROUGE-Lsum F1",
        "bertscore_p":"BERTScore P", "bertscore_r":"BERTScore R", "bertscore_f1":"BERTScore F1",
    })
    final_order = ["Model",
                   "Mean Summary Length (tokens)","Median Summary Length (tokens)",
                   "Reference Mean Length (tokens)","Reference Median Length (tokens)",
                   "ROUGE-1 P","ROUGE-1 R","ROUGE-1 F1","ROUGE-2 P","ROUGE-2 R","ROUGE-2 F1",
                   "ROUGE-Lsum P","ROUGE-Lsum R","ROUGE-Lsum F1","BERTScore P","BERTScore R","BERTScore F1"]

    # Paired summaries
    paired_df_named = paired_df.rename(columns={
        "id":"Article ID",
        "article":"Article",
        "reference":"Reference Summary",
        "ref_len":"Reference Length (tokens)",
        "ft_summary":"FT Summary",
        "ft_len":"FT Length (tokens)",
        "ft_r1_f1":"FT ROUGE-1 F1",
        "ft_r2_f1":"FT ROUGE-2 F1",
        "ft_rlsum_f1":"FT ROUGE-Lsum F1",
        "ft_bs_f1":"FT BERTScore F1",
        "base_summary":"BASE Summary",
        "base_len":"BASE Length (tokens)",
        "base_r1_f1":"BASE ROUGE-1 F1",
        "base_r2_f1":"BASE ROUGE-2 F1",
        "base_rlsum_f1":"BASE ROUGE-Lsum F1",
        "base_bs_f1":"BASE BERTScore F1",
    })

    # Per-example PRF
    metrics_rename = {"id":"Article ID"}
    metrics_rename.update({
        "ft_r1_p":"FT ROUGE-1 P","ft_r1_r":"FT ROUGE-1 R","ft_r1_f1":"FT ROUGE-1 F1",
        "ft_r2_p":"FT ROUGE-2 P","ft_r2_r":"FT ROUGE-2 R","ft_r2_f1":"FT ROUGE-2 F1",
        "ft_rlsum_p":"FT ROUGE-Lsum P","ft_rlsum_r":"FT ROUGE-Lsum R","ft_rlsum_f1":"FT ROUGE-Lsum F1",
        "ft_bs_p":"FT BERTScore P","ft_bs_r":"FT BERTScore R","ft_bs_f1":"FT BERTScore F1",
        "base_r1_p":"BASE ROUGE-1 P","base_r1_r":"BASE ROUGE-1 R","base_r1_f1":"BASE ROUGE-1 F1",
        "base_r2_p":"BASE ROUGE-2 P","base_r2_r":"BASE ROUGE-2 R","base_r2_f1":"BASE ROUGE-2 F1",
        "base_rlsum_p":"BASE ROUGE-Lsum P","base_rlsum_r":"BASE ROUGE-Lsum R","base_rlsum_f1":"BASE ROUGE-Lsum F1",
        "base_bs_p":"BASE BERTScore P","base_bs_r":"BASE BERTScore R","base_bs_f1":"BASE BERTScore F1",
    })
    metrics_df_named = metrics_df.rename(columns=metrics_rename)

    # ===== Write Excel (order: 1) Overview, 2) Full metrics, 3) Paired, 4) Per-example) =====
    with pd.ExcelWriter(combined_xlsx, engine="openpyxl") as xw:
        summary_f1_tbl.to_excel(xw, index=False, sheet_name="Results_Overview")
        final_tbl_named[final_order].to_excel(xw, index=False, sheet_name="All_Metrics_PRF")
        paired_df_named.to_excel(xw, index=False, sheet_name="Paired_Summaries")
        metrics_df_named.to_excel(xw, index=False, sheet_name="Per_Example_PRF")

    print(f"Combined Excel saved: {combined_xlsx}")

finally:
    for p in (tmp_ft_csv, tmp_base_csv):
        try: os.remove(p); print(f"Removed temporary CSV: {p}")
        except Exception as e: print(f"Could not remove temporary CSV {p}: {e}")
    try:
        del ds, bertscore, length_tokenizer
    except Exception:
        pass
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    gc.collect(); print("Done.")


🟩 Device: NVIDIA A100-SXM4-40GB (cc 8.0) | dtype=torch.bfloat16 | maxlen ladder=(3600, 3000, 2200) | batch ladder=[32, 25, 16, 12, 8, 4, 2, 1]


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

...

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

ROUGE per-example: 100%|██████████| 11490/11490 [00:51<00:00, 225.01it/s]


✅ Combined Excel saved: /content/drive/My Drive/ANLP_project/data/models_summarization/cnn_dm_FT_BASE_combined_20250829_150235.xlsx
🧹 Removed temporary CSV: /content/drive/My Drive/ANLP_project/data/models_summarization/tmp_cnn_dm_FT_20250829_150235.csv
🧹 Removed temporary CSV: /content/drive/My Drive/ANLP_project/data/models_summarization/tmp_cnn_dm_BASE_20250829_150235.csv
🏁 Done.


In [3]:
# Disconnect the runtime
from google.colab import runtime
runtime.unassign()

In [ ]:
# # =========================================================
# # Summarize CNN/DM (full test) with your FINE-TUNED merged_model
# # - Batched generation with adaptive OOM backoff
# # - A100 / T4 / CPU compatible (4-bit only on CUDA)
# # - Left padding (pad==eos safety for decoder-only)
# # - Per-example prompt-length slicing
# # - Prints snippet + reference + generated summary via pbar.write(...)
# # - Periodic CSV flush + final ROUGE JSON
# # =========================================================

# import os, json, gc, math
# from datetime import datetime
# from textwrap import shorten

# import torch
# import tqdm
# import pandas as pd
# from datasets import load_dataset
# import evaluate
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# # ── Paths ────────────────────────────────────────────────
# SAVE_DIR ="/content/drive/My Drive/ANLP_project/data/models_summarization/full_llama3_8b_system_prompt_lora_SFT_SWOW_tgt_qkvo_tr7194c_val899c_r16_a32_do0p1_lr0.0001_bs16_ga4"

# RUN_DIR = "/content/drive/My Drive/ANLP_project/full_llama3_8b_system_prompt_lora_SFT_SWOW_tgt_qkvo_tr7194c_val899c_r16_a32_do0p1_lr0.0001_bs16_ga4"
# MODEL_PATH = f"{RUN_DIR}/merged_model"
# os.makedirs(SAVE_DIR, exist_ok=True)
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# # ── Device & dtype plan ─────────────────────────────────
# if torch.cuda.is_available():
#     dev_name = torch.cuda.get_device_name(0)
#     cap_major, cap_minor = torch.cuda.get_device_capability(0)
#     total_mem_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
# else:
#     dev_name, cap_major, cap_minor, total_mem_gb = "CPU", 0, 0, 0

# AMPERE_PLUS = cap_major >= 8            # A100/L4/H100 etc.
# compute_dtype = torch.bfloat16 if AMPERE_PLUS else torch.float16

# # Prompt length ladders (try longer first on bigger GPUs)
# MAXLEN_LADDER = (3600, 3000, 2200) if (AMPERE_PLUS or total_mem_gb >= 30) else (3000, 2200, 1600)

# # Batch size ladder (try larger first, then back off on OOM)
# if total_mem_gb >= 30:       # A100 40/80GB class
#     BS_LADDER = [32, 25, 16, 12, 8, 4, 2, 1]
# elif total_mem_gb >= 22:     # 24GB class
#     BS_LADDER = [8, 4, 3, 2, 1]
# elif total_mem_gb >= 14:     # T4 16GB class
#     BS_LADDER = [8, 4, 3, 2, 1]
# else:                        # small GPU or CPU
#     BS_LADDER = [2, 1] if torch.cuda.is_available() else [1]

# print(f"Device: {dev_name} (cc {cap_major}.{cap_minor}) | dtype={compute_dtype} | "
#       f"maxlen ladder={MAXLEN_LADDER} | batch ladder={BS_LADDER}")

# # ── Quantization (CUDA only) ────────────────────────────
# use_bnb = torch.cuda.is_available()
# bnb_cfg = None
# if use_bnb:
#     bnb_cfg = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_compute_dtype=compute_dtype,
#     )

# # ── Data & metric ───────────────────────────────────────
# # FULL test split (opt-in to trust_remote_code to silence future warning)
# ds = load_dataset("ccdv/cnn_dailymail", "3.0.0", split="test", trust_remote_code=True)
# rouge = evaluate.load("rouge")

# # ── Model & tokenizer ───────────────────────────────────
# tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, use_fast=True)

# # If no explicit pad token, use eos (common for LLaMA). Then enforce LEFT padding.
# if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
#     tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "left"   # critical for decoder-only when pad==eos

# load_kwargs = dict(device_map="auto", low_cpu_mem_usage=True)
# if bnb_cfg is not None:
#     load_kwargs["quantization_config"] = bnb_cfg
# else:
#     # CPU (or no-bnb) fallback
#     load_kwargs["torch_dtype"] = compute_dtype

# model = AutoModelForCausalLM.from_pretrained(MODEL_PATH, **load_kwargs)
# model.eval()

# def build_prompt(article: str) -> str:
#     messages = [
#         {"role": "system", "content": "You are a helpful assistant that writes concise news summaries."},
#         {"role": "user", "content": f"Summarize the following article in 3-4 sentences.\n\nARTICLE:\n{article}\n\nSummary:"}
#     ]
#     return tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

# def _generate_batch(prompts, max_len):
#     """Tokenize a list of prompts and generate summaries as a batch."""
#     inputs = tokenizer(
#         prompts,
#         return_tensors="pt",
#         padding=True,
#         truncation=True,
#         max_length=max_len,
#     )
#     inputs = {k: v.to(model.device) for k, v in inputs.items()}
#     with torch.inference_mode():
#         out = model.generate(
#             **inputs,
#             max_new_tokens=500,
#             do_sample=False,  # deterministic
#             eos_token_id=tokenizer.eos_token_id,
#             pad_token_id=tokenizer.pad_token_id,
#         )

#     attn = inputs["attention_mask"]
#     L_max = inputs["input_ids"].shape[1]
#     left_pad = (tokenizer.padding_side == "left")

#     gens = []
#     for i in range(out.shape[0]):
#         # Start index depends on padding side:
#         start = L_max if left_pad else int(attn[i].sum().item())
#         if start >= out.shape[1]:
#             # extremely rare edge case: no generation happened (EOS immediately)
#             gens.append("")
#             continue
#         gen_ids = out[i, start:]
#         text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
#         gens.append(text)

#     # per-batch cleanup
#     del out, inputs
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#     return gens

# def batched_summarize(articles_iterable, batch_ladder, maxlen_ladder, flush_every=200):
#     """Yield generated summaries with adaptive OOM backoff and periodic flushing."""
#     preds, refs, arts = [], [], []
#     buffer_rows = []
#     current_bs_index = 0      # start with largest batch size
#     current_len_index = 0     # start with largest prompt length

#     total = len(articles_iterable)
#     pbar = tqdm.tqdm(total=total, desc="Generating")

#     idx = 0
#     while idx < total:
#         bs = batch_ladder[current_bs_index]
#         max_len = maxlen_ladder[current_len_index]
#         end = min(idx + bs, total)
#         batch_articles = [articles_iterable[i]["article"] for i in range(idx, end)]
#         batch_refs     = [articles_iterable[i]["highlights"] for i in range(idx, end)]
#         batch_prompts  = [build_prompt(a) for a in batch_articles]

#         try:

#             batch_preds = _generate_batch(batch_prompts, max_len)
#             # # Print batch size
#             # pbar.write(f"\n>>> Processing batch of {len(batch_prompts)} prompts (idx {idx} → {end-1})")

#             # # Print each generated summary for inspection using pbar.write
#             # for i_ex, (a, r, g) in enumerate(zip(batch_articles, batch_refs, batch_preds), start=idx):
#             #     snippet = shorten(a.replace("\n", " "), width=200, placeholder="…")
#             #     pbar.write(f"\n===== Example {i_ex+1} =====")
#             #     pbar.write(f"Article (first 200 chars):\n{snippet}\n")
#             #     pbar.write(f"Reference:\n{r}\n")
#             #     pbar.write(f"Generated Summary:\n{g if g else '[EMPTY]'}")
#             #     pbar.write("===========================================")

#             # accumulate
#             preds.extend(batch_preds)
#             refs.extend(batch_refs)
#             arts.extend(batch_articles)

#             # buffer for periodic flush
#             for a, r, g in zip(batch_articles, batch_refs, batch_preds):
#                 buffer_rows.append((a, r, g))

#             idx = end
#             pbar.update(len(batch_preds))

#             # periodic CSV flush
#             if len(buffer_rows) >= flush_every:
#                 yield ("flush", buffer_rows)
#                 buffer_rows = []

#         except torch.cuda.OutOfMemoryError:
#             if torch.cuda.is_available():
#                 torch.cuda.empty_cache()
#             gc.collect()
#             if current_bs_index < len(batch_ladder) - 1:
#                 current_bs_index += 1
#             elif current_len_index < len(maxlen_ladder) - 1:
#                 current_len_index += 1
#                 current_bs_index = 0
#             else:
#                 ooms = ["[OOM]"] * (end - idx)
#                 preds.extend(ooms)
#                 refs.extend(batch_refs)
#                 arts.extend(batch_articles)
#                 for a, r, g in zip(batch_articles, batch_refs, ooms):
#                     buffer_rows.append((a, r, g))
#                 idx = end
#                 pbar.update(len(ooms))

#     pbar.close()
#     if buffer_rows:
#         yield ("flush", buffer_rows)
#     yield ("done", (arts, refs, preds))

# # ── Output paths ────────────────────────────────────────
# ts = datetime.now().strftime("%Y%m%d_%H%M%S")
# csv_path    = os.path.join(SAVE_DIR, f"cnn_dm_summaries_fulltest_{ts}.csv")
# scores_path = os.path.join(SAVE_DIR, f"cnn_dm_rouge_fulltest_{ts}.json")

# # ── Run generation (stream to CSV in chunks) ────────────
# try:
#     articles_iterable = ds

#     pd.DataFrame(columns=["article", "reference", "summary"]).to_csv(csv_path, index=False, encoding="utf-8")

#     for kind, payload in batched_summarize(articles_iterable, BS_LADDER, MAXLEN_LADDER, flush_every=200):
#         if kind == "flush":
#             rows = payload
#             df_chunk = pd.DataFrame(rows, columns=["article", "reference", "summary"])
#             df_chunk.to_csv(csv_path, mode="a", header=False, index=False, encoding="utf-8")
#         elif kind == "done":
#             arts, refs, preds = payload

#     print(f"Streaming summaries saved to: {csv_path}")

#     scores = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
#     scores_rounded = {k: round(float(v), 4) for k, v in scores.items()}
#     with open(scores_path, "w", encoding="utf-8") as f:
#         json.dump(scores_rounded, f, ensure_ascii=False, indent=2)
#     print("Final ROUGE (full test):")
#     print(scores_rounded)
#     print(f"Saved scores to: {scores_path}")

# finally:
#     try:
#         del ds, preds, refs, arts
#     except Exception:
#         pass
#     try:
#         del model, tokenizer
#     except Exception:
#         pass
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#     gc.collect()
#     print("Freed VRAM (emptied CUDA cache and GC run).")


In [ ]:
# Disconnect the runtime
from google.colab import runtime
runtime.unassign()

In [ ]:
# # =========================================================
# # Summarize CNN/DM (full test) with Base Llama-3-8B Instruct
# # - Batched generation with adaptive OOM backoff
# # - A100 / T4 / CPU compatible (4-bit only on CUDA)
# # - Left padding (pad==eos safety for decoder-only)
# # - Per-example prompt-length slicing
# # - Prints snippet + reference + generated summary via pbar.write(...)
# # - Periodic CSV flush + final ROUGE JSON
# # =========================================================

# import os, json, gc, math
# from datetime import datetime
# from textwrap import shorten

# import torch
# import tqdm
# import pandas as pd
# from datasets import load_dataset
# import evaluate
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# # ── Paths ────────────────────────────────────────────────
# SAVE_DIR = "/content/drive/My Drive/ANLP_project/data/models_summarization/base_model/llama3_8b"


# MODEL_NAME = "meta-llama/Meta-Llama-3-8B-Instruct"
# os.makedirs(SAVE_DIR, exist_ok=True)
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# # ── Device & dtype plan ─────────────────────────────────
# if torch.cuda.is_available():
#     dev_name = torch.cuda.get_device_name(0)
#     cap_major, cap_minor = torch.cuda.get_device_capability(0)
#     total_mem_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)
# else:
#     dev_name, cap_major, cap_minor, total_mem_gb = "CPU", 0, 0, 0

# AMPERE_PLUS = cap_major >= 8            # A100/L4/H100 etc.
# compute_dtype = torch.bfloat16 if AMPERE_PLUS else torch.float16

# # Prompt length ladders (try longer first on bigger GPUs)
# MAXLEN_LADDER = (3600, 3000, 2200) if (AMPERE_PLUS or total_mem_gb >= 30) else (3000, 2200, 1600)

# # Batch size ladder (try larger first, then back off on OOM)
# if total_mem_gb >= 30:       # A100 40/80GB class
#     BS_LADDER = [32, 25, 16, 12, 8, 4, 2, 1]
# elif total_mem_gb >= 22:     # 24GB class
#     BS_LADDER = [8, 4, 3, 2, 1]
# elif total_mem_gb >= 14:     # T4 16GB class
#     BS_LADDER = [8, 4, 3, 2, 1]
# else:                        # small GPU or CPU
#     BS_LADDER = [2, 1] if torch.cuda.is_available() else [1]

# print(f"Device: {dev_name} (cc {cap_major}.{cap_minor}) | dtype={compute_dtype} | "
#       f"maxlen ladder={MAXLEN_LADDER} | batch ladder={BS_LADDER}")

# # ── Quantization (CUDA only) ────────────────────────────
# use_bnb = torch.cuda.is_available()
# bnb_cfg = None
# if use_bnb:
#     bnb_cfg = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_use_double_quant=True,
#         bnb_4bit_compute_dtype=compute_dtype,
#     )

# # ── Data & metric ───────────────────────────────────────
# # FULL test split (opt-in to trust_remote_code to silence future warning)
# ds = load_dataset("ccdv/cnn_dailymail", "3.0.0", split="test", trust_remote_code=True)
# rouge = evaluate.load("rouge")

# # ── Model & tokenizer ───────────────────────────────────
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

# # If no explicit pad token, use eos (common for LLaMA). Then enforce LEFT padding.
# if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
#     tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "left"   # critical for decoder-only when pad==eos

# load_kwargs = dict(device_map="auto", low_cpu_mem_usage=True)
# if bnb_cfg is not None:
#     load_kwargs["quantization_config"] = bnb_cfg
# else:
#     # CPU (or no-bnb) fallback
#     load_kwargs["torch_dtype"] = compute_dtype

# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, **load_kwargs)
# model.eval()

# def build_prompt(article: str) -> str:
#     messages = [
#         {"role": "system", "content": "You are a helpful assistant that writes concise news summaries."},
#         {"role": "user", "content": f"Summarize the following article in 3-4 sentences.\n\nARTICLE:\n{article}\n\nSummary:"}
#     ]
#     return tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

# def _generate_batch(prompts, max_len):
#     """Tokenize a list of prompts and generate summaries as a batch."""
#     inputs = tokenizer(
#         prompts,
#         return_tensors="pt",
#         padding=True,
#         truncation=True,
#         max_length=max_len,
#     )
#     inputs = {k: v.to(model.device) for k, v in inputs.items()}
#     with torch.inference_mode():
#         out = model.generate(
#             **inputs,
#             max_new_tokens=500,
#             do_sample=False,  # deterministic
#             eos_token_id=tokenizer.eos_token_id,
#             pad_token_id=tokenizer.pad_token_id,
#         )

#     attn = inputs["attention_mask"]
#     L_max = inputs["input_ids"].shape[1]
#     left_pad = (tokenizer.padding_side == "left")

#     gens = []
#     for i in range(out.shape[0]):
#         # Start index depends on padding side:
#         start = L_max if left_pad else int(attn[i].sum().item())
#         if start >= out.shape[1]:
#             # extremely rare edge case: no generation happened (EOS immediately)
#             gens.append("")
#             continue
#         gen_ids = out[i, start:]
#         text = tokenizer.decode(gen_ids, skip_special_tokens=True).strip()
#         gens.append(text)

#     # per-batch cleanup
#     del out, inputs
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#     return gens

# def batched_summarize(articles_iterable, batch_ladder, maxlen_ladder, flush_every=200):
#     """Yield generated summaries with adaptive OOM backoff and periodic flushing."""
#     preds, refs, arts = [], [], []
#     buffer_rows = []
#     current_bs_index = 0      # start with largest batch size
#     current_len_index = 0     # start with largest prompt length

#     total = len(articles_iterable)
#     pbar = tqdm.tqdm(total=total, desc="Generating")

#     idx = 0
#     while idx < total:
#         bs = batch_ladder[current_bs_index]
#         max_len = maxlen_ladder[current_len_index]
#         end = min(idx + bs, total)
#         batch_articles = [articles_iterable[i]["article"] for i in range(idx, end)]
#         batch_refs     = [articles_iterable[i]["highlights"] for i in range(idx, end)]
#         batch_prompts  = [build_prompt(a) for a in batch_articles]

#         try:

#             batch_preds = _generate_batch(batch_prompts, max_len)

#             # # Print batch size
#             # pbar.write(f"\n>>> Processing batch of {len(batch_prompts)} prompts (idx {idx} → {end-1})")

#             # # Print each generated summary for inspection using pbar.write
#             # for i_ex, (a, r, g) in enumerate(zip(batch_articles, batch_refs, batch_preds), start=idx):
#             #     snippet = shorten(a.replace("\n", " "), width=200, placeholder="…")
#             #     pbar.write(f"\n===== Example {i_ex+1} =====")
#             #     pbar.write(f"Article (first 200 chars):\n{snippet}\n")
#             #     pbar.write(f"Reference:\n{r}\n")
#             #     pbar.write(f"Generated Summary:\n{g if g else '[EMPTY]'}")
#             #     pbar.write("===========================================")

#             # accumulate
#             preds.extend(batch_preds)
#             refs.extend(batch_refs)
#             arts.extend(batch_articles)

#             # buffer for periodic flush
#             for a, r, g in zip(batch_articles, batch_refs, batch_preds):
#                 buffer_rows.append((a, r, g))

#             idx = end
#             pbar.update(len(batch_preds))

#             # periodic CSV flush
#             if len(buffer_rows) >= flush_every:
#                 yield ("flush", buffer_rows)
#                 buffer_rows = []

#         except torch.cuda.OutOfMemoryError:
#             if torch.cuda.is_available():
#                 torch.cuda.empty_cache()
#             gc.collect()
#             if current_bs_index < len(batch_ladder) - 1:
#                 current_bs_index += 1
#             elif current_len_index < len(maxlen_ladder) - 1:
#                 current_len_index += 1
#                 current_bs_index = 0
#             else:
#                 ooms = ["[OOM]"] * (end - idx)
#                 preds.extend(ooms)
#                 refs.extend(batch_refs)
#                 arts.extend(batch_articles)
#                 for a, r, g in zip(batch_articles, batch_refs, ooms):
#                     buffer_rows.append((a, r, g))
#                 idx = end
#                 pbar.update(len(ooms))

#     pbar.close()
#     if buffer_rows:
#         yield ("flush", buffer_rows)
#     yield ("done", (arts, refs, preds))

# # ── Output paths ────────────────────────────────────────
# ts = datetime.now().strftime("%Y%m%d_%H%M%S")
# csv_path    = os.path.join(SAVE_DIR, f"cnn_dm_summaries_fulltest_{ts}.csv")
# scores_path = os.path.join(SAVE_DIR, f"cnn_dm_rouge_fulltest_{ts}.json")

# # ── Run generation (stream to CSV in chunks) ────────────
# try:
#     articles_iterable = ds

#     pd.DataFrame(columns=["article", "reference", "summary"]).to_csv(csv_path, index=False, encoding="utf-8")

#     for kind, payload in batched_summarize(articles_iterable, BS_LADDER, MAXLEN_LADDER, flush_every=200):
#         if kind == "flush":
#             rows = payload
#             df_chunk = pd.DataFrame(rows, columns=["article", "reference", "summary"])
#             df_chunk.to_csv(csv_path, mode="a", header=False, index=False, encoding="utf-8")
#         elif kind == "done":
#             arts, refs, preds = payload

#     print(f"Streaming summaries saved to: {csv_path}")

#     scores = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
#     scores_rounded = {k: round(float(v), 4) for k, v in scores.items()}
#     with open(scores_path, "w", encoding="utf-8") as f:
#         json.dump(scores_rounded, f, ensure_ascii=False, indent=2)
#     print("Final ROUGE (full test):")
#     print(scores_rounded)
#     print(f"Saved scores to: {scores_path}")

# finally:
#     try:
#         del ds, preds, refs, arts
#     except Exception:
#         pass
#     try:
#         del model, tokenizer
#     except Exception:
#         pass
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#     gc.collect()
#     print("Freed VRAM (emptied CUDA cache and GC run).")


In [ ]:
# Disconnect the runtime
from google.colab import runtime
runtime.unassign()